<a href="https://colab.research.google.com/github/ThisIsFarhan/GPT2-FineTuning/blob/main/gpt2_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset, load_dataset

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("distilbert/distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilbert/distilgpt2")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving scene_descriptions_dataset.json to scene_descriptions_dataset.json


In [ ]:
data_id = "scene_descriptions_dataset.json"

In [ ]:
def prepare_train_data(data_id):
    data = load_dataset("json", data_files=data_id, split="train")
    data_df = data.to_pandas()
    data_df["text"] = data_df[["input", "output"]].apply(lambda x: x["input"] + ".\n" + x["output"], axis=1)
    data = Dataset.from_pandas(data_df[['text']])
    return data

In [ ]:
dataset = prepare_train_data(data_id)

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
dataset['text'][0]

'left_input: phone, chair, lamp\nright_input: cabinet, person, human\nup_input: mouse, glasses\nbottom_input: laptop, shelf.\nIn the left side area, there were a phone, a chair, and a lamp. In the right area, there were a cabinet, a person, and a human. In the above area, there were a mouse and a glasses. a laptop and a shelf were detected in the below section.'

In [ ]:
def tokenize_and_add(data):
  tokens = tokenizer(data['text'], truncation=True, padding='max_length', max_length=180)
  return {
      'input_ids':tokens['input_ids'],
      'labels':tokens['input_ids']
  }

tokenized_dataset = dataset.map(tokenize_and_add, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'labels'],
    num_rows: 1000
})

In [ ]:
training_args = TrainingArguments(
    output_dir="content/gpt2-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir = "./logs",
    logging_steps = 10,
    save_steps=10,
    fp16=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: farhanak128 (farhanak128-comsats-university-islamabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,6.988100
20,1.331600
30,0.611600
40,0.440600
50,0.345700
60,0.312700
70,0.301700
80,0.284100
90,0.268100
100,0.269500


TrainOutput(global_step=375, training_loss=0.46794338258107504, metrics={'train_runtime': 486.6043, 'train_samples_per_second': 6.165, 'train_steps_per_second': 0.771, 'total_flos': 137793208320000.0, 'train_loss': 0.46794338258107504, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./gpt2-FYP')
tokenizer.save_pretrained('./gpt2-FYP')

('./gpt2-FYP/tokenizer_config.json',
 './gpt2-FYP/special_tokens_map.json',
 './gpt2-FYP/vocab.json',
 './gpt2-FYP/merges.txt',
 './gpt2-FYP/added_tokens.json')

In [ ]:
model_path = './gpt2-FYP'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

def response(input):
  inputs = tokenizer.encode(input,return_tensors="pt")
  outputs = model.generate(inputs, max_length=150, num_return_sequences=1)
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return response

In [ ]:
response('left_input: phone, chair, lamp\nright_input: cabinet')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'left_input: phone, chair, lamp\nright_input: cabinet, lamp\nup_input: lamp, lamp, person.\na phone, a chair, and a lamp were observed left side. a cabinet, a lamp, and a person were observed right side. a lamp, a lamp, and a person were observed upper.'

In [ ]:
!zip -r /content/gpt2_finetuned_model.zip /content/gpt2-FYP

  adding: content/gpt2-FYP/ (stored 0%)
  adding: content/gpt2-FYP/model.safetensors (deflated 7%)
  adding: content/gpt2-FYP/tokenizer_config.json (deflated 56%)
  adding: content/gpt2-FYP/vocab.json (deflated 68%)
  adding: content/gpt2-FYP/generation_config.json (deflated 24%)
  adding: content/gpt2-FYP/merges.txt (deflated 53%)
  adding: content/gpt2-FYP/config.json (deflated 51%)
  adding: content/gpt2-FYP/special_tokens_map.json (deflated 74%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/gpt2_finetuned_model.zip /content/drive/MyDrive/

Mounted at /content/drive
